In [1]:
import pandas as pd
df = pd.read_csv("data.csv")
df.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0',
       'Country name', 'Year', 'Life Ladder', 'Log GDP per capita',
       'Social support', 'Healthy life expectancy at birth',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Positive affect', 'Negative affect',
       'Confidence in national government', 'cont'],
      dtype='object')

In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Veri setini yükle
df = pd.read_csv('data.csv')

# İndeks sütunlarını temizle
cols_to_drop = [col for col in df.columns if 'Unnamed: 0' in col]
df = df.drop(columns=cols_to_drop, errors='ignore')

# Avrupa verisini filtrele ('EU' kıta kodu)
df_eu = df[df['cont'] == 'EU'].copy()

# 5 metriği ve yeni kategori sütunlarını tanımla
metrics = {
    'Log GDP per capita': 'GDP_Level',
    'Social support': 'Social_Level',
    'Healthy life expectancy at birth': 'Health_Level',
    'Freedom to make life choices': 'Freedom_Level', # Yeni metrik
    'Life Ladder': 'Happiness_Level'
}
levels = ['Düşük', 'Orta', 'Yüksek'] # Kategori etiketleri

# Her metriği Avrupa verisinin tersillerine göre kategorize et
for col, new_col in metrics.items():
    p_33 = df_eu[col].quantile(0.333)
    p_66 = df_eu[col].quantile(0.667)
    bins = [-np.inf, p_33, p_66, np.inf]
    df_eu[new_col] = pd.cut(
        df_eu[col],
        bins=bins,
        labels=levels,
        right=True,
        include_lowest=True
    )

# --- Akış Tablolarını Oluşturma (4 akış) ---

flows = {}

# Flow 1: GDP -> Sosyal Destek
flows['GDP_to_Social'] = df_eu.groupby(['GDP_Level', 'Social_Level']).size().reset_index(name='value')
flows['GDP_to_Social'].columns = ['source_level', 'target_level', 'value']

# Flow 2: Sosyal Destek -> Sağlıklı Yaşam Süresi
flows['Social_to_Health'] = df_eu.groupby(['Social_Level', 'Health_Level']).size().reset_index(name='value')
flows['Social_to_Health'].columns = ['source_level', 'target_level', 'value']

# Flow 3: Sağlıklı Yaşam Süresi -> Özgürlük
flows['Health_to_Freedom'] = df_eu.groupby(['Health_Level', 'Freedom_Level']).size().reset_index(name='value')
flows['Health_to_Freedom'].columns = ['source_level', 'target_level', 'value']

# Flow 4: Özgürlük -> Mutluluk
flows['Freedom_to_Happiness'] = df_eu.groupby(['Freedom_Level', 'Happiness_Level']).size().reset_index(name='value')
flows['Freedom_to_Happiness'].columns = ['source_level', 'target_level', 'value']


# --- Sankey Diyagramı İçin Düğüm (Node) ve Bağlantı (Link) Tanımlama ---

# Diyagramda kullanılacak Türkçe aşama etiketleri
stage_map = {
    'GDP_Level': 'GDP (Kişi Başı)',
    'Social_Level': 'Sosyal Destek',
    'Health_Level': 'Sağlıklı Yaşam Süresi',
    'Freedom_Level': 'Yaşam Seçim Özgürlüğü',
    'Happiness_Level': 'Mutluluk (Life Ladder)'
}
ordered_stages = ['GDP_Level', 'Social_Level', 'Health_Level', 'Freedom_Level', 'Happiness_Level']

# Düğüm Listesi
nodes = []
all_levels = ['Düşük', 'Orta', 'Yüksek'] # Kategoriler
for stage_key in ordered_stages:
    for level in all_levels:
        node_name = f"{stage_map[stage_key]} - {level}"
        if node_name not in nodes:
            nodes.append(node_name)

# Düğüm etiketinden indekse eşleme
label_to_index = {label: i for i, label in enumerate(nodes)}

# Bağlantı Listesi
source = []
target = []
value = []

# Akışları bağlantı listesine ekleyen yardımcı fonksiyon
def add_flow(df_flow, source_col_key, target_col_key):
    source_name = stage_map[source_col_key]
    target_name = stage_map[target_col_key]
    for index, row in df_flow.iterrows():
        source_label = f"{source_name} - {row['source_level']}"
        target_label = f"{target_name} - {row['target_level']}"
        source.append(label_to_index[source_label])
        target.append(label_to_index[target_label])
        value.append(row['value'])

# Akışları ekle (4 akış)
add_flow(flows['GDP_to_Social'], 'GDP_Level', 'Social_Level')
add_flow(flows['Social_to_Health'], 'Social_Level', 'Health_Level')
add_flow(flows['Health_to_Freedom'], 'Health_Level', 'Freedom_Level')
add_flow(flows['Freedom_to_Happiness'], 'Freedom_Level', 'Happiness_Level')

# Sankey Diyagramını Oluşturma ve Kaydetme
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes,
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
    )
)])

fig.update_layout(
    title_text="Avrupa İçin GDP'den Mutluluğa 5 Aşamalı Akış Şeması (Sankey Diyagramı)",
    font_size=10
)

fig.show()

/tmp/ipykernel_7987/3135938140.py:43: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_7987/3135938140.py:47: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_7987/3135938140.py:51: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_7987/3135938140.py:55: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=F

In [7]:
df.to_csv("dataa.csv")